# Import modules 

In [12]:
import pandas as pd
import qgrid
import easygui
import glob

# Helper functions

In [7]:
def count_digits(input):
    return int(math.log10(input))+1


# Import data

In [19]:
# import data

print("Select a file inside a folder that contains the raw grade file.")
target_dir = easygui.diropenbox("Select a folder that containts the raw check-in files", 
  "Select a folder that containts the raw check-in files")

file_data = []
for file in glob.glob(target_dir+"/Log*.txt"):
    df = pd.read_csv(file)
    file_data.append(df)

data = pd.concat(file_data)


# data['IDcheck'] = data['StudentID'].apply(count_digits)

Select a file inside a folder that contains the raw grade file.
